In [105]:
import sys 

sys.path.append("/home/davi/PycharmProjects/Reconhecimento-de-Padroes")

from sklearn.model_selection import KFold
from Strings import string
from sklearn.model_selection import train_test_split
from numpy import zeros, identity, array, concatenate, \
    ones, ravel, dot, where, multiply, sign, mean, exp, var, subtract
from src.Utils.utils import get_accuracy, get_data
from scipy.spatial.distance import cdist

from pandas import DataFrame
import cvxopt
import cvxopt.solvers

In [106]:
data = get_data("column_2C_weka.arff", type="arff")
data = DataFrame(data)

In [107]:
number_lines = data.shape[0]
number_columns = data.shape[1]
x = array(data, ndmin=2)[:, :number_columns-1]
y =   array(array(data, ndmin=2)[:, number_columns-1], ndmin=2).T

indices = where(y == b'Abnormal')
y[indices] = -1

indices = where(y == b'Normal')
y[indices] = 1

train_size = .8
test_size = .2


In [108]:
def kernel(x_train, y_train):
    
    number_lines, number_columns = x_train.shape
    # Create the kernel matrix
    
    # P matrix
    linear_kernel = (dot(x_train, x_train.T) + 1)
    gausian = zeros((number_lines, number_lines))
    for i in range(number_lines):
        for j in range(number_lines):
            gausian[i][j] = dot(x_train[i]- x_train[j],(x_train[i]- x_train[j]).T) 
    
    # print(gausian)
    sigma = 1
    gausian_kernel = exp((-1./(sigma**2))*gausian)
    kernel = array(dot(y_train, y_train.T)*gausian_kernel, dtype=float)
    # print(kernel.shape)
    P = cvxopt.matrix(kernel, tc='d')
    # print("shape p:" , P.size)
    
    # Q marix
    Q = cvxopt.matrix(-1.*ones((number_lines, 1)), tc='d')
    # print("sahpe Q: ", Q.size)
    
    # A matrix
    A = cvxopt.matrix(array(y_train.T, dtype=float), tc='d')
    # print("shape A: ", A.size)
    
    # b matrix
    b = cvxopt.matrix(zeros((1,  1)), tc='d')
    # print("sahpe b:" , b.size)
    
    # G matrix
    G = cvxopt.matrix(-1.*ones((number_lines, number_lines)), tc='d')
    # print("shape G", G.size)
    
    # H matrix 
    H =  cvxopt.matrix(-1.*zeros((number_lines, 1)), tc='d')
    # print("shape H:", H.size)
    
    
    try:
        solution = cvxopt.solvers.qp(P, Q ,A=A, b=b, kktsolver='ldl')
    except ValueError as error:
        print(error)

    alphas = ravel(solution['x'])
    weights = dot(x_train.T, array(alphas, ndmin=2).T*y_train)
    aux = dot(x_train, weights)
    bias = y_train - aux.T
    penalization = dot(alphas*y_train,linear_kernel) + bias
    
    return alphas, weights

In [109]:
def train(x_train, y_train):
    solution = kernel(x_train, y_train)
    return solution

In [110]:
def predict(u):
    return sign(u)



In [111]:
def test(weights, x_test, y_test):
    # 
    # aux = dot(x_test, weights)
    # bias = y_test - aux
    
    out = dot(x_test, weights)
    y_output = predict(out) 
    accuracy = get_accuracy(y_output, y_test)
    
    # for i in range(y_test.shape[0]):
    #     print("output: ", y_output[i], "Expected: ", y_test[i])
    return accuracy

In [112]:
acc = []

for realization in range(10):

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
    alphas, weights = train(x_train, y_train)
    accuracy = test(weights, x_test, y_test)
    acc.append(accuracy)
    print("realization: ", realization)
    print("acc: ", accuracy)
    
print("Media de acerto: ", mean(acc))

realization:  0
acc:  0.6774193548387096
realization:  1
acc:  0.7258064516129032
realization:  2
acc:  0.6935483870967742
realization:  3
acc:  0.7096774193548387
realization:  4
acc:  0.6290322580645161
realization:  5
acc:  0.6774193548387096
realization:  6
acc:  0.6935483870967742
realization:  7
acc:  0.7903225806451613
realization:  8
acc:  0.6774193548387096
realization:  9
acc:  0.7419354838709677
Media de acerto:  0.7016129032258065
